# Bereitstellen von Informationen aus einem PDF-Dokument

Dieses Notebook stellt Informationen aus einer PDF-Datei als spezielle Datenbank, einem sogenannten `VectorStore` zur Verfügung.
Weitere Informationen findest Du in [diesem Notebook](./index.ipynb).


## Nötige Software installieren

Die folgenden beiden Zellen installieren ein paar benötigte Python-Bibliotheken und laden ein paar vertrauliche Konfigurationsdaten (API-Schlüssel): 

In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet chromadb  dotenv langchain langgraph langsmith langchain_chroma langchain-community langchain_openai langchain-unstructured langchain-docling unstructured-client unstructured "unstructured[pdf]" python-magic
%load_ext dotenv

In [ ]:
%dotenv /home/archive/FerienkursKI/.env

Wir laden nun die benötigten Python-Pakete, um PDF-Dateien zu verarbeiten und die Datenbank zu erstellen.

In [ ]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from docling.chunking import HybridChunker, HierarchicalChunker
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import chromadb
import os

Zur Verarbeitung der PDF-Dateien verwenden Docling.

In [ ]:
# Pfad der PDF-Datei
pdf_path = "./data/Koalitionsvertrag-2025-1.pdf" 

chunker = HybridChunker()
loader = DoclingLoader(file_path=pdf_path, chunker=chunker)
docling = loader.load()

Wir teilen den Text in "Chunks" auf, um die Verarbeitung zu erleichtern.

In [ ]:
# Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(docling)

Unser VectorStore kommt mit den von Docling gelieferten Metadaten nicht klar, daher vereinfachen wir die Metadaten.

In [ ]:
# Initialisiere Chunks und Metadaten für den VectorStore
for chunk in chunks:
    meta = {
            "source": chunk.metadata['source'],
            "heading": chunk.metadata['dl_meta']['headings'][0],
            "page": chunk.metadata['dl_meta']['doc_items'][0]['prov'][0]['page_no']
        }
    chunk.metadata = meta



In [ ]:
chunks[0]

Document(metadata={'source': './data/Koalitionsvertrag-2025-1.pdf', 'heading': 'Verantwortung für Deutschland', 'page': 1}, page_content='Verantwortung für Deutschland\nKoalitionsvertrag zwischen CDU, CSU und SPD\n21. Legislaturperiode')

Nun erzeugen wir den VectorStore und speichern ihn in der Chroma-Datenbank.

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

chromadb.configure(allow_reset=True)

client = chromadb.PersistentClient(path="./chroma")
#client.list_collections() 

In [ ]:
vector_store = Chroma(
    client=client,
    collection_name="Koalitionsvertrag",
    embedding_function=embeddings,
)

In [ ]:
ids = vector_store.add_documents(documents=chunks)

Die folgende Zelle zeigt beispielhaft, wie Du den VectorStore verwenden kannst.

In [ ]:
retriever = vector_store.as_retriever()
docs = retriever.invoke("Was steht im Koalitionsvertrag zum Thema Bildung?")

In [ ]:
docs